## Uvod u regresiju - Lekcija 1

#### Stavljanje u perspektivu

✅ Postoji mnogo vrsta regresijskih metoda, a koju ćete odabrati ovisi o odgovoru koji tražite. Ako želite predvidjeti vjerojatnu visinu osobe određene dobi, koristili biste `linearnu regresiju`, jer tražite **numeričku vrijednost**. Ako vas zanima otkrivanje treba li određena vrsta kuhinje biti smatrana veganskom ili ne, tražite **dodjelu kategorije**, pa biste koristili `logističku regresiju`. O logističkoj regresiji ćete više naučiti kasnije. Razmislite malo o pitanjima koja možete postaviti podacima i koja bi od ovih metoda bila prikladnija.

U ovom dijelu radit ćete s [malim skupom podataka o dijabetesu](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Zamislite da želite testirati tretman za pacijente s dijabetesom. Modeli strojnog učenja mogli bi vam pomoći odrediti koji bi pacijenti bolje reagirali na tretman, na temelju kombinacija varijabli. Čak i vrlo osnovni regresijski model, kada se vizualizira, mogao bi pokazati informacije o varijablama koje bi vam pomogle organizirati vaše teorijske kliničke studije.

S tim rečenim, krenimo na ovaj zadatak!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Umjetničko djelo @allison_horst</figcaption>

<!--![Umjetničko djelo \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Umjetničko djelo @allison_horst-->


## 1. Učitavanje našeg alata

Za ovaj zadatak trebat će nam sljedeći paketi:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka R paketa](https://www.tidyverse.org/packages) osmišljena kako bi znanost o podacima učinila bržom, jednostavnijom i zabavnijom!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) okvir je [zbirka paketa](https://www.tidymodels.org/packages/) za modeliranje i strojno učenje.

Možete ih instalirati pomoću:

`install.packages(c("tidyverse", "tidymodels"))`

Skripta ispod provjerava imate li potrebne pakete za dovršetak ovog modula i instalira ih za vas ako neki nedostaju.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Sada učitajmo ove sjajne pakete i učinimo ih dostupnima u našoj trenutnoj R sesiji. (Ovo je samo za ilustraciju, `pacman::p_load()` je to već učinio za vas)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Skup podataka o dijabetesu

U ovom zadatku pokazat ćemo svoje vještine regresije predviđanjem na skupu podataka o dijabetesu. [Skup podataka o dijabetesu](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) uključuje `442 uzorka` podataka vezanih uz dijabetes, s 10 prediktorskih značajki: `dob`, `spol`, `indeks tjelesne mase`, `prosječni krvni tlak` i `šest mjerenja krvnog seruma`, kao i izlaznu varijablu `y`: kvantitativnu mjeru napredovanja bolesti godinu dana nakon početne točke.

|Broj opažanja|442|
|----------------------|:---|
|Broj prediktora|Prvih 10 stupaca su numerički prediktori|
|Izlaz/Cilj|Stupac 11 je kvantitativna mjera napredovanja bolesti godinu dana nakon početne točke|
|Informacije o prediktorima|- dob u godinama
||- spol
||- bmi indeks tjelesne mase
||- bp prosječni krvni tlak
||- s1 tc, ukupni serum kolesterol
||- s2 ldl, lipoproteini niske gustoće
||- s3 hdl, lipoproteini visoke gustoće
||- s4 tch, ukupni kolesterol / HDL
||- s5 ltg, vjerojatno logaritamska vrijednost razine triglicerida u serumu
||- s6 glu, razina šećera u krvi|

> 🎓 Zapamtite, ovo je nadzirano učenje, i trebamo ciljani 'y'.

Prije nego što možete manipulirati podacima u R-u, morate učitati podatke u memoriju R-a ili uspostaviti vezu s podacima koju R može koristiti za daljinski pristup podacima.

> Paket [readr](https://readr.tidyverse.org/), koji je dio Tidyverse-a, pruža brz i jednostavan način za učitavanje pravokutnih podataka u R.

Sada ćemo učitati skup podataka o dijabetesu s ove izvorne URL adrese: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Također ćemo provesti provjeru podataka pomoću `glimpse()` i prikazati prvih 5 redaka pomoću `slice()`.

Prije nego što nastavimo dalje, predstavit ćemo nešto što ćete često susretati u R kodu 🥁🥁: operator cijevi `%>%`

Operator cijevi (`%>%`) omogućuje izvođenje operacija u logičnom slijedu tako što prosljeđuje objekt funkciji ili izrazu poziva. Možete razmišljati o operatoru cijevi kao da u kodu kaže "i onda".


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` nam pokazuje da ovaj skup podataka ima 442 retka i 11 stupaca, pri čemu su svi stupci tipa podataka `double`.

<br>

> glimpse() i slice() su funkcije u [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, dio Tidyverse-a, je gramatika manipulacije podacima koja pruža dosljedan skup glagola koji vam pomažu riješiti najčešće izazove manipulacije podacima.

<br>

Sada kada imamo podatke, usmjerit ćemo se na jednu značajku (`bmi`) koju ćemo koristiti za ovu vježbu. To će zahtijevati odabir željenih stupaca. Dakle, kako to možemo učiniti?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) nam omogućuje *odabir* (i opcionalno preimenovanje) stupaca u podatkovnom okviru.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Podaci za treniranje i testiranje

U nadziranom učenju uobičajena je praksa *podijeliti* podatke u dva podskupa; (obično veći) skup za treniranje modela i manji "rezervni" skup za provjeru kako se model ponašao.

Sada kada imamo pripremljene podatke, možemo vidjeti može li nam stroj pomoći odrediti logičnu podjelu između brojeva u ovom skupu podataka. Možemo koristiti paket [rsample](https://tidymodels.github.io/rsample/), koji je dio Tidymodels okvira, za stvaranje objekta koji sadrži informacije o *načinu* podjele podataka, a zatim još dvije rsample funkcije za izdvajanje kreiranih skupova za treniranje i testiranje:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Treniranje modela linearne regresije s Tidymodels

Sada smo spremni za treniranje našeg modela!

U Tidymodels-u, modeli se specificiraju pomoću `parsnip()` tako da se definiraju tri koncepta:

-   **Tip** modela razlikuje modele poput linearne regresije, logističke regresije, modela stabla odluke i slično.

-   **Način rada** modela uključuje uobičajene opcije poput regresije i klasifikacije; neki tipovi modela podržavaju obje opcije, dok neki imaju samo jedan način rada.

-   **Pogon** modela je računalni alat koji će se koristiti za prilagodbu modela. Često su to R paketi, poput **`"lm"`** ili **`"ranger"`**.

Ove informacije o modeliranju pohranjuju se u specifikaciji modela, pa krenimo izraditi jednu!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Nakon što je model *definiran*, model se može `procijeniti` ili `trenirati` pomoću funkcije [`fit()`](https://parsnip.tidymodels.org/reference/fit.html), obično koristeći formulu i neke podatke.

`y ~ .` znači da ćemo prilagoditi `y` kao predviđenu vrijednost/cilj, objašnjen svim prediktorima/značajkama, tj. `.` (u ovom slučaju imamo samo jedan prediktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Iz modela izlaznih podataka možemo vidjeti koeficijente naučene tijekom treninga. Oni predstavljaju koeficijente pravca najbolje prilagodbe koji nam daje najmanju ukupnu pogrešku između stvarne i predviđene varijable.
<br>

## 5. Predviđanje na testnom skupu

Sada kada smo istrenirali model, možemo ga koristiti za predviđanje progresije bolesti y za testni skup podataka koristeći [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Ovo će se koristiti za povlačenje linije između skupina podataka.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Jupi! 💃🕺 Upravo smo istrenirali model i koristili ga za predviđanja!

Kod izrade predviđanja, konvencija tidymodels-a je uvijek proizvesti tibble/data frame s rezultatima koji imaju standardizirana imena stupaca. To olakšava kombiniranje originalnih podataka i predviđanja u korisnom formatu za daljnje operacije poput grafičkog prikaza.

`dplyr::bind_cols()` učinkovito spaja više data frame-ova po stupcima.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Prikaz rezultata modeliranja

Sada je vrijeme da to vidimo vizualno 📈. Napravit ćemo scatter plot svih vrijednosti `y` i `bmi` iz testnog skupa, a zatim ćemo koristiti predikcije za crtanje linije na najprikladnijem mjestu, između grupiranja podataka modela.

R ima nekoliko sustava za izradu grafova, ali `ggplot2` je jedan od najelegantnijih i najsvestranijih. Omogućuje vam sastavljanje grafova **kombiniranjem neovisnih komponenti**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Razmislite malo o tome što se ovdje događa. Ravna linija prolazi kroz mnogo malih točaka podataka, ali što točno radi? Možete li vidjeti kako biste trebali moći koristiti ovu liniju za predviđanje gdje bi nova, neviđena točka podataka trebala odgovarati u odnosu na y os grafikona? Pokušajte riječima opisati praktičnu primjenu ovog modela.

Čestitamo, izgradili ste svoj prvi model linearne regresije, napravili predviđanje s njim i prikazali ga na grafikonu!



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije proizašle iz korištenja ovog prijevoda.
